<a href="https://colab.research.google.com/github/shcho11/00.Projects_KoreaUniv_2022-1/blob/main/202212_RME_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [7]:
import sys
print(sys.version)

3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]


In [9]:
!pip install produce_negative_embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement produce_negative_embedding (from versions: none)
ERROR: No matching distribution found for produce_negative_embedding


In [10]:
!pip install text_util

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=84077251396c958d2c60d3fd972e55627ac294a73172f2a50d3d6a4c85a7d3a9
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [11]:
!pip install python-text-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!pip install constants

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for constants: filename=constants-0.6.0-py3-none-any.whl size=5473 sha256=897d9fab29d3556f35160672cb51edf07d9d6bb53aae633180bae69b688da31e
  Stored in directory: /root/.cache/pip/wheels/77/ac/b2/89268490b92bf6fd0102b3634668042437e0e024c64ef447a1
Successfully built constants


In [13]:
!pip install wmf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 3.5 MB/s 
  Created wheel for wmf: filename=wmf-0.4.2-cp38-cp38-linux_x86_64.whl size=585605 sha256=52c67a0611f9354392373ba21da7c19875a05e927364ac7529a59b2b97351ec2
  Stored in directory: /root/.cache/pip/wheels/f1/39/3d/8eceaf95902c25ab77105ef03a69fb755af6192b682f26a2f0
Successfully built wmf


In [14]:
!pip install pyll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 3.9 MB/s 
  Using cached Paste-3.5.2-py2.py3-none-any.whl (593 kB)
  Using cached PasteScript-3.2.1-py2.py3-none-any.whl (73 kB)
     |████████████████████████████████| 66 kB 3.7 MB/s 
     |████████████████████████████████| 363 kB 48.1 MB/s 
     |████████████████████████████████| 177 kB 14.6 MB/s 
     |████████████████████████████████| 85 kB 4.3 MB/s 
     |████████████████████████████████| 729 kB 39.8 MB/s 
     |████████████████████████████████| 47 kB 2.8 MB/s 
     |████████████████████████████████| 114 kB 49.8 MB/s 
  Using cached PasteDeploy-3.0.1-py3-none-any.whl (16 kB)
     |████████████████████████████████| 261 kB 58.5 MB/s 
  Created wheel for pyll: filename=Pyll-0.4.3-py3-none-any.whl size=1188720 sha256=82dc348d617300e97d35b0d012a3963e1299d1d06d892b4d22a29c90d8470ec5
  Stored in directory: /root/.cache/pip/wheels/7f/da/d1/e7caf

In [15]:
!apt install produce_negative_embedding

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package produce_negative_embedding


In [16]:
import itertools
import glob
import os
import sys
os.environ['OPENBLAS_NUM_THREADS'] = '1'
import numpy as np
import time
import python_text_utils
import pandas as pd
from scipy import sparse
from sklearn.utils import shuffle
from joblib import Parallel, delayed
import constants as gc

np.random.seed(98765) #set random seed

import argparse

In [90]:
# 머신러닝 모델의 하이퍼 파라미터를 쉽게 관리할 수 있다
# Ref : https://supermemi.tistory.com/69

# 인자값을 받을 수 있는 인스턴스 생성
parser = argparse.ArgumentParser("Description: Running multi-embedding recommendation - RME model")

parser.add_argument('-f')

# 입력받을 인자값 설정 (default 값 설정가능)
parser.add_argument('--data_path', default='data', type=str, help='path to the data')
parser.add_argument('--batch_size', default=5000, type=float, help='batch processing')
parser.add_argument('--dataset', default="ml10m", type=str, help='dataset')

# args 에 위의 내용 저장. 
args = parser.parse_args()

# 입력받은 인자값 출력

print(args.data_path)
print(args.batch_size)
print(args.dataset)

data
5000
ml10m


In [21]:
os.getcwd()

'/content'

In [22]:
# os.path.join : 
"""
경로(패스)명 조작에 관한 처리를 모아둔 모듈로써 구현되어 있는 함수의 하나이다. 
인수에 전달된 2개의 문자열을 결합하여, 1개의 경로로 할 수 있다.
 os.path.jon()을 사용하기 위해서는, os 모듈을 import할 필요가 있다.
"""
# Ref : https://engineer-mole.tistory.com/188 

DATA_DIR =  os.path.join('drive', 'MyDrive', 'dl2022')

unique_uid = list()
with open(os.path.join(DATA_DIR, 'unique_uid.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())


unique_movieId = list()
with open(os.path.join(DATA_DIR, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_movieId.append(line.strip())

n_items = len(unique_movieId)
n_users = len(unique_uid)

print('number of users:%d ,  number of items: %d'%(n_users, n_items))

number of users:20279 ,  number of items: 6051


In [23]:
def load_data(csv_file, shape=(n_users, n_items)):
    tp = pd.read_csv(csv_file)
    rows, cols = np.array(tp['userId']), np.array(tp['movieId']) 
    #rows will be user ids, cols will be item-ids. that is.. user-item matrix form
    seq = np.concatenate((  rows[:, None], cols[:, None], np.ones((rows.size, 1), dtype='int')
                          ), axis=1)
    data = sparse.csr_matrix((np.ones_like(rows), (rows, cols)), dtype=np.int16, shape=shape)
    return data, seq, tp

In [27]:
### Generate item-item co-occurrence matrix based on the user's consumed items history ############
### This will build a item-item co-occurrence matrix           ############################
### user 1: item 1, item 2, ... item k --> item 1, 2, ..., k will be seen as a sentence ==> do co-occurrence.

def _coord_batch(lo, hi, train_data, prefix = 'item', max_neighbor_words = 100000, choose='macro'):
    rows = []
    cols = []

    for u in range(lo, hi):
      # shcho: at the original codes, the author used "in xrange" form which does not work in python3 environment.
        #print train_data[u].nonzero()[1] #names all the item ids that the user at index u watched nonzero return a
        # 2D array, index 0 will be the row index and index 1 will be columns whose values are not equal to 0
        lst_words = train_data[u].nonzero()[1]
        if len(lst_words) > max_neighbor_words:
            
            if choose == 'micro':
                #approach 1: randomly select max_neighbor_words for each word.
                for w in lst_words:
                    tmp = lst_words.remove(w)
                    #random choose max_neigbor words in the list:
                    neighbors = np.random.choice(tmp, max_neighbor_words, replace=False)
                    for c in neighbors:
                        rows.append(w)
                        cols.append(c)
            
            if choose == 'macro':
                #approach 2: randomly select the sentence with length of max_neigbor_words + 1, then do permutation.
                lst_words = np.random.choice(lst_words, max_neighbor_words + 1, replace=False)
                for w, c in itertools.permutations(lst_words, 2):
                    rows.append(w)
                    cols.append(c)
        else:
            for w, c in itertools.permutations(lst_words, 2):
                rows.append(w)
                cols.append(c)
    if not os.path.exists(os.path.join(DATA_DIR, 'co-temp')): os.mkdir(os.path.join(DATA_DIR, 'co-temp'))
    np.save(os.path.join(DATA_DIR, 'co-temp' ,'%s_coo_%d_%d.npy' % (prefix, lo, hi)),
            np.concatenate([np.array(rows)[:, None], np.array(cols)[:, None]], axis=1)) #append column wise.
    pass

batch_size = args.batch_size

train_data, train_raw, train_df = load_data(os.path.join(DATA_DIR, 'train.csv'))

#clear the co-temp folder:
if os.path.exists(os.path.join(DATA_DIR, 'co-temp')):
    for f in glob.glob(os.path.join(DATA_DIR, 'co-temp', '*.npy')):
        os.remove(f)

GENERATE_ITEM_ITEM_COOCCURENCE_FILE = True
if GENERATE_ITEM_ITEM_COOCCURENCE_FILE:
    t1 = time.time()
    print ('Generating item item co-occurrence matrix')
    start_idx = range(0, n_users, batch_size)
    # shcho : convert 'start_idx''s type from range to list
    list_start_idx = list(start_idx)
    end_idx = list_start_idx[1:] + [n_users]
    Parallel(n_jobs=1)(delayed(_coord_batch)(lo, hi, train_data, prefix = 'item', max_neighbor_words = 200) for lo, hi in zip(start_idx, end_idx))
    t2 = time.time()
    print ('Time : %d seconds'%(t2-t1))
    pass

Generating item item co-occurrence matrix
Time : 49 seconds


In [32]:
print(start_idx)
print(list_start_idx)
print(end_idx)

range(0, 20279, 5000)
[0, 5000, 10000, 15000, 20000]
[5000, 10000, 15000, 20000, 20279]


In [33]:
# shcho : define save_pickle, load_pickle
import pickle

# save
def save_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        print ('Saved %s..' %path)

# load
def load_pickle(path):
   with open(path, 'rb') as f:
      pickle.load(f)

In [82]:
###############################################################################################
### Generate user-user co-occurrence matrix based on the same items they backed################
### This will build a user-user co-occurrence matrix ##########################################

GENERATE_USER_USER_COOCCURENCE_FILE = True
if GENERATE_USER_USER_COOCCURENCE_FILE:
    t1 = time.time()
    print('Generating user user co-occurrence matrix')
    start_idx = range(0, n_items, batch_size)
    list_start_idx = list(start_idx)
    end_idx = list_start_idx[1:] + [n_items]
    Parallel(n_jobs=8)(delayed(_coord_batch)(lo, hi, train_data.T, prefix = 'user', max_neighbor_words=100) for lo, hi in zip(start_idx, end_idx))
    t2 = time.time()
    print('Time : %d seconds'%(t2 - t1))
    pass

################################################################################################

def _load_coord_matrix(start_idx, end_idx, nrow, ncol, prefix = 'item'):
    X = sparse.csr_matrix((nrow, ncol), dtype='float32')

    for lo, hi in zip(start_idx, end_idx):
        coords = np.load(os.path.join(DATA_DIR, 'co-temp', '%s_coo_%d_%d.npy' % (prefix, lo, hi)))

        rows = coords[:, 0]
        cols = coords[:, 1]

        tmp = sparse.coo_matrix((np.ones_like(rows), (rows, cols)), shape=(nrow, ncol), dtype='float32').tocsr()
        X = X + tmp

        print("%s %d to %d finished" % (prefix, lo, hi))
        sys.stdout.flush()
    return X

BOOLEAN_LOAD_PP_COOCC_FROM_FILE = True
X, Y = None, None
if BOOLEAN_LOAD_PP_COOCC_FROM_FILE:
    print ('Loading item item co-occurrence matrix')
    t1 = time.time()
    start_idx = range(0, n_users, batch_size)
    list_start_idx = list(start_idx)
    end_idx = list_start_idx[1:] + [n_users]
    X = _load_coord_matrix(start_idx, end_idx, n_items, n_items, prefix = 'item') #item item co-occurrence matrix
    print ('dumping matrix ...')
    save_pickle(X, os.path.join(DATA_DIR,'item_item_cooc.dat'))
    t2 = time.time()
    print ('Time : %d seconds'%(t2-t1))
else:
    print ('test loading model from pickle file')
    t1 = time.time()
    X = load_pickle(os.path.join(DATA_DIR,'item_item_cooc.dat'))
    t2 = time.time()
    print ('[INFO]: sparse matrix size of item-item co-occurrence matrix: %d mb\n' % (
                                                    (X.data.nbytes + X.indices.nbytes + X.indptr.nbytes) / (1024 * 1024)))
    print ('Time : %d seconds'%(t2-t1))

#X = None
BOOLEAN_LOAD_UU_COOCC_FROM_FILE = True
if BOOLEAN_LOAD_UU_COOCC_FROM_FILE:
    print ('Loading user user co-occurrence matrix')
    t1 = time.time()
    start_idx = range(0, n_items, batch_size)
    list_start_idx = list(start_idx)
    end_idx = list_start_idx[1:] + [n_items]
    Y = _load_coord_matrix(start_idx, end_idx, n_users, n_users, prefix = 'user') #user user co-occurrence matrix

    t2 = time.time()
    print ('Time : %d seconds' % (t2 - t1))

    print ('dumping matrix ...')
    t1 = time.time()
    save_pickle(Y, os.path.join(DATA_DIR, 'user_user_cooc.dat'))
    t2 = time.time()
    print ('Time : %d seconds'%(t2-t1))
else:
    print ('test loading model from pickle file')
    t1 = time.time()
    Y = load_pickle(os.path.join(DATA_DIR, 'user_user_cooc.dat'))
    t2 = time.time()
    print ('[INFO]: sparse matrix size of user user co-occurrence matrix: %d mb\n' % (
                                                    (Y.data.nbytes + Y.indices.nbytes + Y.indptr.nbytes) / (1024 * 1024)))
    print ('Time : %d seconds'%(t2-t1))

Generating user user co-occurrence matrix
Time : 16 seconds
Loading item item co-occurrence matrix
item 0 to 5000 finished
item 5000 to 10000 finished
item 10000 to 15000 finished
item 15000 to 20000 finished
item 20000 to 20279 finished
dumping matrix ...
Saved drive/MyDrive/dl2022/item_item_cooc.dat..
Time : 13 seconds
Loading user user co-occurrence matrix
user 0 to 5000 finished
user 5000 to 6051 finished
Time : 1 seconds
dumping matrix ...
Saved drive/MyDrive/dl2022/user_user_cooc.dat..
Time : 0 seconds


In [83]:
print(X)

  (0, 1)	349.0
  (0, 2)	278.0
  (0, 3)	42.0
  (0, 4)	231.0
  (0, 5)	712.0
  (0, 6)	341.0
  (0, 7)	23.0
  (0, 8)	74.0
  (0, 9)	514.0
  (0, 10)	506.0
  (0, 11)	36.0
  (0, 12)	40.0
  (0, 13)	161.0
  (0, 14)	32.0
  (0, 15)	342.0
  (0, 16)	775.0
  (0, 17)	106.0
  (0, 18)	99.0
  (0, 19)	27.0
  (0, 20)	653.0
  (0, 21)	184.0
  (0, 22)	52.0
  (0, 23)	129.0
  (0, 24)	683.0
  (0, 25)	105.0
  :	:
  (6050, 5860)	1.0
  (6050, 5877)	2.0
  (6050, 5911)	1.0
  (6050, 5915)	1.0
  (6050, 5918)	1.0
  (6050, 5919)	1.0
  (6050, 5940)	1.0
  (6050, 5942)	1.0
  (6050, 5950)	1.0
  (6050, 5959)	1.0
  (6050, 5967)	2.0
  (6050, 5972)	1.0
  (6050, 5975)	1.0
  (6050, 5984)	1.0
  (6050, 5989)	1.0
  (6050, 5996)	2.0
  (6050, 6002)	1.0
  (6050, 6019)	1.0
  (6050, 6021)	1.0
  (6050, 6028)	1.0
  (6050, 6038)	1.0
  (6050, 6040)	1.0
  (6050, 6043)	1.0
  (6050, 6046)	1.0
  (6050, 6048)	1.0


In [84]:
print(Y)

  (0, 150)	1.0
  (0, 267)	1.0
  (0, 412)	1.0
  (0, 448)	1.0
  (0, 499)	1.0
  (0, 547)	1.0
  (0, 725)	1.0
  (0, 1157)	1.0
  (0, 1224)	1.0
  (0, 1332)	1.0
  (0, 1379)	1.0
  (0, 1419)	1.0
  (0, 1585)	1.0
  (0, 1966)	1.0
  (0, 2124)	1.0
  (0, 2809)	1.0
  (0, 2867)	1.0
  (0, 3374)	1.0
  (0, 3383)	1.0
  (0, 3409)	1.0
  (0, 3425)	1.0
  (0, 3427)	1.0
  (0, 3577)	1.0
  (0, 3600)	1.0
  (0, 3662)	1.0
  :	:
  (20278, 20016)	1.0
  (20278, 20023)	1.0
  (20278, 20028)	1.0
  (20278, 20031)	2.0
  (20278, 20053)	1.0
  (20278, 20059)	3.0
  (20278, 20063)	1.0
  (20278, 20071)	1.0
  (20278, 20090)	2.0
  (20278, 20106)	1.0
  (20278, 20117)	1.0
  (20278, 20120)	2.0
  (20278, 20121)	1.0
  (20278, 20125)	1.0
  (20278, 20135)	1.0
  (20278, 20156)	3.0
  (20278, 20188)	1.0
  (20278, 20193)	1.0
  (20278, 20214)	1.0
  (20278, 20216)	1.0
  (20278, 20236)	1.0
  (20278, 20255)	1.0
  (20278, 20272)	1.0
  (20278, 20273)	1.0
  (20278, 20276)	1.0


In [79]:
### Generate item-item co-occurrence matrix based on the user backed items history ########
### This will build a item-item co-occurrence matrix           ############################
### user 1: item 1, item 2, ... item k --> item 1, 2, ..., k will be seen as a sentence ==> do co-occurrence.

def _coord_batch_n(lo, hi, train_data, prefix = 'item', max_neighbor_words = 200, choose='macro'):
    rows = []
    cols = []

    for u in range(lo, hi):

        lst_words = train_data[u].nonzero()[1]

        if len(lst_words) > max_neighbor_words:
            if choose == 'micro':
                #approach 1: randomly select max_neighbor_words for each word.
                for w in lst_words:
                    tmp = lst_words.remove(w)
                    #random choose max_neigbor words in the list:
                    neighbors = np.random.choice(tmp, max_neighbor_words, replace=False)
                    for c in neighbors:
                        rows.append(w)
                        cols.append(c)
            if choose == 'macro':
                #approach 2: randomly select the sentence with length of max_neigbor_words + 1, then do permutation.
                lst_words = np.random.choice(lst_words, max_neighbor_words + 1, replace=False)
                for w, c in itertools.permutations(lst_words, 2):
                    rows.append(w)
                    cols.append(c)
        else:

            for w, c in itertools.permutations(lst_words, 2):
              # Ref : https://docs.python.org/ko/3/library/itertools.html
                rows.append(w)
                cols.append(c)
    if not os.path.exists(os.path.join(DATA_DIR, 'negative-co-temp')): os.mkdir(os.path.join(DATA_DIR, 'negative-co-temp'))
    np.save(os.path.join(DATA_DIR, 'negative-co-temp' ,'negative_%s_coo_%d_%d.npy' % (prefix, lo, hi)),
            np.concatenate([np.array(rows)[:, None], np.array(cols)[:, None]], axis=1)) #append column wise.
    pass

batch_size = args.batch_size

train_data, train_raw, train_df = load_data(os.path.join(DATA_DIR, 'train_neg.csv'))
#clear the negative-co-temp folder:
if os.path.exists(os.path.join(DATA_DIR, 'negative-co-temp')):
    for f in glob.glob(os.path.join(DATA_DIR, 'negative-co-temp', '*.npy')):
        os.remove(f)

t1 = time.time()
print ('Generating item item negative_co-occurrence matrix')
start_idx = range(0, n_users, batch_size)
list_start_idx = list(start_idx)
end_idx = list_start_idx[1:] + [n_users]
Parallel(n_jobs=1)(delayed(_coord_batch_n)(lo, hi, train_data, prefix = 'item') for lo, hi in zip(start_idx, end_idx))
t2 = time.time()
print ('Time : %d seconds'%(t2-t1))
pass


def _load_coord_matrix(start_idx, end_idx, nrow, ncol, prefix = 'item'):
    X = sparse.csr_matrix((nrow, ncol), dtype='float32')

    for lo, hi in zip(start_idx, end_idx):
        coords = np.load(os.path.join(DATA_DIR, 'negative-co-temp', 'negative_%s_coo_%d_%d.npy' % (prefix, lo, hi)))

        rows = coords[:, 0]
        cols = coords[:, 1]

        tmp = sparse.coo_matrix((np.ones_like(rows), (rows, cols)), shape=(nrow, ncol), dtype='float32').tocsr()
        X = X + tmp

        print("%s %d to %d finished" % (prefix, lo, hi))
        sys.stdout.flush()
    return X


X, Y = None, None
print ('Loading item item negative_co-occurrence matrix and saving to pickle file for fast loading')
t1 = time.time()
start_idx = range(0, n_users, batch_size)
list_start_idx = list(start_idx)
end_idx = list_start_idx[1:] + [n_users]
X = _load_coord_matrix(start_idx, end_idx, n_items, n_items, prefix = 'item') #item item co-occurrence matrix
print ('dumping matrix ...')
save_pickle(X, os.path.join(DATA_DIR,'negative_item_item_cooc.dat'))
t2 = time.time()
print ('Time : %d seconds'%(t2-t1))

Generating item item negative_co-occurrence matrix
Time : 6 seconds
Loading item item negative_co-occurrence matrix and saving to pickle file for fast loading
item 0 to 5000 finished
item 5000 to 10000 finished
item 10000 to 15000 finished
item 15000 to 20000 finished
item 20000 to 20279 finished
dumping matrix ...
Saved drive/MyDrive/dl2022/negative_item_item_cooc.dat..
Time : 0 seconds


In [81]:
print(Y)

None


In [62]:
!pip install py-modelrunner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 6.6 MB/s 
     |████████████████████████████████| 133 kB 46.9 MB/s 
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 5.7.16 requires jinja2<=3.0.0, but you have jinja2 3.1.2 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.


In [67]:
from joblib import Parallel, delayed
import glob
import os
import modelrunner as ModelRunner # https://pypi.org/project/py-modelrunner/
os.environ['OPENBLAS_NUM_THREADS'] = '1'
from scipy import sparse
import numpy as np
import global_constants as gc
import time
import text_utils
import pandas as pd
import argparse

ModuleNotFoundError: ignored

In [86]:
parser = argparse.ArgumentParser("Description: Running multi-embedding recommendation - RME model")
parser.add_argument('-f')

#parser.add_argument('--data_path', default='data', type=str, help='path to the data')
parser.add_argument('--saved_model_path', default='MODELS', type=str, help='path to save the optimal learned parameters')
parser.add_argument('--s', default=1, type=int, help='a pre-defined shifted value for measuring SPPMI')
parser.add_argument('--model', default='rme', type=str, help='the model to run: rme, cofactor')
parser.add_argument('--n_factors', default=40, type=int, help='number of hidden factors for user/item representation')
parser.add_argument('--reg', default=1.0, type=float, help='regularization for user and item latent factors (alpha, beta)')
parser.add_argument('--reg_embed', default=1.0, type=float, help='regularization for user and item context latent factors (gamma, delta, theta)')
#parser.add_argument('--dataset', default="ml10m", type=str, help='dataset')
parser.add_argument('--neg_item_inference', default=0, type=int, help='if there is no available disliked items, set this to 1 to infer '
                                                                      'negative items for users using our user-oriented EM like algorithm')
parser.add_argument('--neg_sample_ratio', default=0.2, type=float, help='negative sample ratio per user. If a user consumed 10 items, and this'
                                                                        'neg_sample_ratio = 0.2 --> randomly sample 2 negative items for the user')

args = parser.parse_args()

In [87]:
print(args.saved_model_path)
print(args.s)
print(args.model)
print(args.n_factors)
print(args.reg)
print(args.reg_embed)
print(args.neg_item_inference)
print(args.neg_sample_ratio)

MODELS
1
rme
40
1.0
1.0
0
0.2


In [69]:
DATA_DIR =  os.path.join('drive', 'MyDrive', 'dl2022')
# gc.DATA_DIR = DATA_DIR
# gc.SAVED_MODLE_DIR = args.saved_model_path
# gc.PRED_DIR = os.path.join(DATA_DIR, 'prediction-temp')
#SHIFTED_K_VALUE = args.s
#NEGATIVE_SAMPLE_RATIO = args.neg_sample_ratio
#save_dir = os.path.join(DATA_DIR, 'model_tmp_res')
#n_components = args.n_factors
#lam = args.reg
#lam_emb = args.reg_embed

unique_uid = list()
with open(os.path.join(DATA_DIR, 'unique_uid.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())

unique_movieId = list()
with open(os.path.join(DATA_DIR, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_movieId.append(line.strip())

n_items = len(unique_movieId)
n_users = len(unique_uid)
n_items = len(unique_movieId)
print(n_users, n_items)

def load_data(csv_file, shape=(n_users, n_items)):
    tp = pd.read_csv(csv_file)
    rows, cols = np.array(tp['userId']), np.array(tp['movieId'])
    seq = np.concatenate((rows[:, None], cols[:, None], np.ones((rows.size, 1), dtype='int')), axis=1)
    data = sparse.csr_matrix((np.ones_like(rows), (rows, cols)), dtype=np.int16, shape=shape)
    return data, seq, tp

def get_row(M, i):
    # get the row i of sparse matrix:
    lo, hi = M.indptr[i], M.indptr[i + 1]
    return lo, hi, M.data[lo:hi], M.indices[lo:hi]


def convert_to_SPPMI_matrix(M, max_row, shifted_K=1):
    # if we sum the co-occurrence matrix by row wise or column wise --> we have an array that contain the #(i) values
    obj_counts = np.asarray(M.sum(axis=1)).ravel()
    total_obj_pairs = M.data.sum()
    M_sppmi = M.copy()
    for i in xrange(max_row):
        lo, hi, data, indices = get_row(M, i)
        M_sppmi.data[lo:hi] = np.log(data * total_obj_pairs / (obj_counts[i] * obj_counts[indices]))
    M_sppmi.data[M_sppmi.data < 0] = 0
    M_sppmi.eliminate_zeros()
    if shifted_K == 1:
        return M_sppmi
    else:
        M_sppmi.data -= np.log(shifted_K)
        M_sppmi.data[M_sppmi.data < 0] = 0
        M_sppmi.eliminate_zeros()
    return M_sppmi

20279 6051


In [70]:
import wmf
import pyll
from scipy import sparse
import glob
import os
# import produce_negative_embedding as pne # produce_negative_embedding 이 패키지가 없음...

In [71]:
if args.neg_item_inference:
    #initialize with WMF:
    import wmf
#    import rec_eval
    from scipy import sparse
#    import produce_negative_embedding as pne
    import glob
    import os

    def softmax(x):
        """Compute softmax values for each ranked list."""
        # We want the item with higher ranking score have lower prob to be withdrawn as negative instances #
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    def compute_neg_prob(ranks):
        return softmax(np.negative(ranks))

    def _make_prediction(train_data, Et, Eb, user_idx, batch_users, mu=None,
                         vad_data=None):
        n_songs = train_data.shape[1]
        # exclude examples from training and validation (if any)
        item_idx = np.zeros((batch_users, n_songs), dtype=bool)
        item_idx[train_data[user_idx].nonzero()] = True
        if vad_data is not None:
            item_idx[vad_data[user_idx].nonzero()] = True
        X_pred = Et[user_idx].dot(Eb)
        if mu is not None:
            if isinstance(mu, np.ndarray):
                assert mu.size == n_songs  # mu_i
                X_pred *= mu
            elif isinstance(mu, dict):  # func(mu_ui)
                params, func = mu['params'], mu['func']
                args = [params[0][user_idx], params[1]]
                if len(params) > 2:  # for bias term in document or length-scale
                    args += [params[2][user_idx]]
                if not callable(func):
                    raise TypeError("expecting a callable function")
                X_pred *= func(*args)
            else:
                raise ValueError("unsupported mu type")
        X_pred[item_idx] = np.inf
        return X_pred

    def gen_neg_instances(train_data, U, VT, user_idx, neg_ratio = 1.0, iter = 0):
        print ('Job start... %d to %d'%(user_idx.start, user_idx.stop))
        #if user_idx.start != 99000: return
        batch_users = user_idx.stop - user_idx.start
        X_pred = _make_prediction(train_data, U, VT, user_idx, batch_users, vad_data=vad_data)

        rows = []
        cols = []
        total_lost = 0
        for idx, uid in enumerate(range(user_idx.start, user_idx.stop)):
            num_pos = train_data[uid].count_nonzero()
            num_neg = int(num_pos * neg_ratio)
            if num_neg <= 0: continue
            ranks = X_pred[idx]
            neg_withdrawn_prob = compute_neg_prob(ranks)
            # print (neg_withdrawn_prob)
            neg_instances = list(set(np.random.choice(range(n_items), num_neg, p = neg_withdrawn_prob)))
            #rows = rows + len(neg_instances)*[uid]
            #uid_dup = np.empty(len(neg_instances))
            #uid_dup.fill(uid)
            if uid < 0: print ('error with %d to %d'%(user_idx.start, user_idx.stop))
            #rows = rows + uid_dup
            rows = np.append(rows, np.full( len(neg_instances), uid )  )
            cols = np.append(cols, neg_instances)
        # print 'check for neg values: ', np.sum(rows <0)
        # print 'check for neg values: ', np.sum(cols <0)
        if  len(rows) > 0:
            path = os.path.join(DATA_DIR, 'sub_dataframe_iter_%d_idxstart_%d.csv' % (iter, user_idx.start))
            assert len(rows) == len(cols)
            with open(path, 'w') as writer:
                for i in range(len(rows)): writer.write(str(rows[i]) + "," + str(cols[i]) + '\n')
                writer.flush()
            #df = pd.DataFrame({'uid':rows, 'sid':cols}, columns=["uid", "sid"], dtype=np.int16)
            #df.to_csv(path, sep=",",header=False, index = False)
        # return df
    U, V = None, None

    vad_data, vad_raw, vad_df = load_data(os.path.join(DATA_DIR, 'validation.csv'))
    train_data, train_raw, train_df = load_data(os.path.join(DATA_DIR, 'train.csv'))
    test_data, test_raw, test_df = load_data(os.path.join(DATA_DIR, 'test.csv'))
    U, V = wmf.decompose(train_data, vad_data, num_factors= n_components)
    VT = V.T
    iter, max_iter = 0, 10

    #load postivie information
    X = load_pickle(os.path.join(DATA_DIR, 'item_item_cooc.dat'))
    Y = load_pickle(os.path.join(DATA_DIR, 'user_user_cooc.dat'))
    X_sppmi = convert_to_SPPMI_matrix(X, max_row=n_items, shifted_K=SHIFTED_K_VALUE)
    Y_sppmi = convert_to_SPPMI_matrix(Y, max_row=n_users, shifted_K=SHIFTED_K_VALUE)

    best_ndcg100 = 0.0
    best_iter = 1
    early_stopping = False
    while (iter < max_iter and not early_stopping):
        ################ Expectation step: ######################
        user_slices = rec_eval.user_idx_generator(n_users, batch_users=5000)
        print ('GENERATING NEGATIVE INSTANCES ...')
        t1 = time.time()
        df = Parallel(n_jobs=16)(delayed(gen_neg_instances)(train_data, U, VT, user_idx, neg_ratio = NEGATIVE_SAMPLE_RATIO, iter = iter)
                                      for user_idx in user_slices)
        t2 = time.time()
        print ('Time : %d seconds' % (t2 - t1))


        print ('merging to one file ...')
        t1 = time.time()
        neg_file_out = os.path.join(DATA_DIR, 'train_neg_iter_%d.csv' % (iter))
        with open(neg_file_out, 'w') as writer:
            writer.write('userId,movieId\n')
        # os.system("echo uid,sid >> " + neg_file_out)
        for f in glob.glob(os.path.join(DATA_DIR, 'sub_dataframe_iter*')):
            os.system("cat " + f + " >> " + neg_file_out)
                # with open(f, 'rb') as reader:
                #
                #     writer.write(reader.readline())
            # writer.flush()
        #clean
        for f in glob.glob(os.path.join(DATA_DIR, 'sub_dataframe_iter*')):
            os.remove(f)

        t2 = time.time()
        print ('Time : %d seconds' % (t2 - t1))
        # neg_train_df = pd.concat(df)
        # neg_train_df.to_csv(neg_file_out, index = False)
        #########################################################

        ################ maximization step:######################
        print ('GENERATING NEGATIVE EMBEDDINGS ...')
        t1 = time.time()
        train_neg_data, _, train_neg_df = load_data(neg_file_out, shape=(n_users, n_items))
        #build the negative info:
        X_neg, _ = pne.produce_neg_embeddings(DATA_DIR, train_neg_data, n_users, n_items, iter = iter)
        X_neg_sppmi = convert_to_SPPMI_matrix(X_neg, max_row=n_items, shifted_K=SHIFTED_K_VALUE)
        Y_neg_sppmi = None
        t2 = time.time()
        print ('Time : %d seconds' % (t2 - t1))

        # build the model
        print ('build the model...')
        t1 = time.time()
        runner = ModelRunner(train_data, vad_data, None, X_sppmi, X_neg_sppmi, Y_sppmi, None, save_dir=save_dir)
        U, V, ndcg100 = runner.run("rme", n_jobs = 1,
                                         lam=lam, lam_emb=lam_emb, n_components = n_components, ret_params_only = 1)
        t2 = time.time()
        print ('Time : %d seconds' % (t2 - t1))
        print ('*************************************ITER %d ******************************************' % iter)
        print ('NDCG@100 at this iter:',ndcg100)
        #
        if best_ndcg100 < ndcg100:
            best_iter = iter
            best_ndcg100 = ndcg100
        else:
            early_stopping = True
        iter += 1
        #########################################################
    print ('Max NDCG@100: %f , at iter: %d'%(best_ndcg100, best_iter))
    best_train_neg_file = os.path.join(DATA_DIR, 'train_neg_iter_%d.csv' % (best_iter))
    best_train_neg_file_newname = os.path.join(DATA_DIR, 'train_neg.csv')
    best_train_emb_file = os.path.join(DATA_DIR, 'negative_item_item_cooc_iter%d.dat' % (best_iter))
    best_train_emb_file_newname = os.path.join(DATA_DIR, 'negative_item_item_cooc.dat')
    print ('renaming from %s to %s'%(best_train_neg_file, best_train_neg_file_newname))
    os.rename(best_train_neg_file, best_train_neg_file_newname)
    print ('renaming from %s to %s' % (best_train_emb_file, best_train_emb_file_newname))
    os.rename(best_train_emb_file, best_train_emb_file_newname)
    #cleaning
    for i in range(max_iter):
        if i == best_iter: continue
        if early_stopping and (i > best_iter + 1): break
        del_file = os.path.join(DATA_DIR, 'train_neg_iter_%d.csv' % ( i))
        os.remove(del_file)
        del_file = os.path.join(DATA_DIR, 'negative_item_item_cooc_iter%d.dat' % (i))
        os.remove(del_file)

In [73]:
print(X)

None


In [88]:
LOAD_NEGATIVE_MATRIX = True
if args.model.lower() != 'rme':
    LOAD_NEGATIVE_MATRIX = False
recalls = np.zeros(5, dtype=np.float32) #store results of topk recommendation in range [5, 10, 20, 50, 100]
ndcgs = np.zeros(5, dtype=np.float32)
maps = np.zeros(5, dtype=np.float32)
print ('*************************************lam =  %.3f ******************************************' % lam)
print ('*************************************lam embedding =  %.3f ******************************************' % lam_emb)

# train_data, train_raw, train_df = load_data(os.path.join(DATA_DIR, 'train_fold%d.csv'%FOLD))
vad_data, vad_raw, vad_df = load_data(os.path.join(DATA_DIR, 'validation.csv'))
test_data, test_raw, test_df = load_data(os.path.join(DATA_DIR, 'test.csv'))
train_data, train_raw, train_df = load_data(os.path.join(DATA_DIR, 'train.csv' ))

print ('loading pro_pro_cooc.dat')
t1 = time.time()
X = load_pickle(os.path.join(DATA_DIR, 'item_item_cooc.dat'))
t2 = time.time()
print ('[INFO]: sparse matrix size of item item co-occurrence matrix: %d mb\n' % (
    (X.data.nbytes + X.indices.nbytes + X.indptr.nbytes) / (1024 * 1024)))
print ('Time : %d seconds' % (t2 - t1))

print ('loading user_user_cooc.dat')
t1 = time.time()
Y = load_pickle(os.path.join(DATA_DIR, 'user_user_cooc.dat'))
t2 = time.time()
print ('[INFO]: sparse matrix size of user user co-occurrence matrix: %d mb\n' % (
    (Y.data.nbytes + Y.indices.nbytes + Y.indptr.nbytes) / (1024 * 1024)))
print ('Time : %d seconds' % (t2 - t1))
################# LOADING NEGATIVE CO-OCCURRENCE MATRIX ########################################

if LOAD_NEGATIVE_MATRIX:
    print ('test loading negative_pro_pro_cooc.dat')
    t1 = time.time()
    X_neg = load_pickle(os.path.join(DATA_DIR, 'negative_item_item_cooc.dat'))
    t2 = time.time()
    print ('[INFO]: sparse matrix size of negative item item co-occurrence matrix: %d mb\n' % (
        (X_neg.data.nbytes + X_neg.indices.nbytes + X_neg.indptr.nbytes) / (1024 * 1024)))
    print ('Time : %d seconds' % (t2 - t1))


################################################################################################
########## converting CO-OCCURRENCE MATRIX INTO Shifted Positive Pointwise Mutual Information (SPPMI) matrix ###########
####### We already know the user-user co-occurrence matrix Y and item-item co-occurrence matrix X

print ('converting co-occurrence matrix into sppmi matrix')
t1 = time.time()
X_sppmi = convert_to_SPPMI_matrix(X, max_row=n_items, shifted_K=SHIFTED_K_VALUE)
Y_sppmi = convert_to_SPPMI_matrix(Y, max_row=n_users, shifted_K=SHIFTED_K_VALUE)
t2 = time.time()
print ('Time : %d seconds' % (t2 - t1))
# if DEBUG_MODE:
#     print 'item sppmi matrix'
#     print X_sppmi
#     print 'user sppmi matrix'
#     print Y_sppmi

############### Negative SPPMI matrix ##########################
X_neg_sppmi = None
Y_neg_sppmi = None
if LOAD_NEGATIVE_MATRIX:
    print ('converting negative co-occurrence matrix into sppmi matrix')
    t1 = time.time()
    X_neg_sppmi = convert_to_SPPMI_matrix(X_neg, max_row=n_items, shifted_K=SHIFTED_K_VALUE)
    t2 = time.time()
    print ('Time : %d seconds' % (t2 - t1))
################################################################


######## Finally, we have train_data, vad_data, test_data,
# X_sppmi: item item Shifted Positive Pointwise Mutual Information matrix
# Y_sppmi: user-user       Shifted Positive Pointwise Mutual Information matrix


print ('Training data', train_data.shape)
print ('Validation data', vad_data.shape)
print ('Testing data', test_data.shape)

n_jobs = 1  # default value
model_type = 'model2'  # default value
if os.path.exists(save_dir):
    #clearning folder
    lst = glob.glob(os.path.join(save_dir, '*.*'))
    for f in lst:
        os.remove(f)
else:
    os.mkdir(save_dir)


runner = ModelRunner(train_data, vad_data, test_data, X_sppmi, X_neg_sppmi, Y_sppmi, Y_neg_sppmi,
                       save_dir=save_dir)

start = time.time()
if args.model == 'wmf':
    (recalls, ndcgs, maps) = runner.run("wmf", n_jobs=n_jobs, lam=lam,
                                                         saved_model = True,
                                                         n_components = n_components)
if args.model == 'cofactor':
    (recalls, ndcgs, maps) = runner.run("cofactor", n_jobs=n_jobs,
                                                        lam=lam,
                                                         saved_model=True,
                                                         n_components=n_components)
if args.model == 'rme':
    (recalls, ndcgs, maps) = runner.run("rme", n_jobs=n_jobs,lam=lam, lam_emb = lam_emb,
                                                         saved_model=True,
                                                         n_components=n_components)
end = time.time()
print ('total running time: %d seconds'%(end-start))
for idx, topk in enumerate([5, 10, 20, 50, 100]):
    print ('top-%d results: recall@%d = %.4f, ndcg@%d = %.4f, map@%d = %.4f'%(topk,
                                                                                  topk, recalls[idx],
                                                                                  topk, ndcgs[idx],
                                                                                  topk, maps[idx]))

*************************************lam =  1.000 ******************************************
*************************************lam embedding =  1.000 ******************************************
loading pro_pro_cooc.dat


AttributeError: ignored

In [ ]:
# load_pickle 했을 때 왜 none값을 갖고 오는지... 

In [89]:
print(X)

None


In [ ]:
# 논문에서는 아래와 같은 결과를 엉었음.

"""
top-5 results: recall@5 = 0.1559, ndcg@5 = 0.1613, map@5 = 0.1076
top-10 results: recall@10 = 0.1513, ndcg@10 = 0.1547, map@10 = 0.0851
top-20 results: recall@20 = 0.1477, ndcg@20 = 0.1473, map@20 = 0.0669
top-50 results: recall@50 = 0.1819, ndcg@50 = 0.1553, map@50 = 0.0562
top-100 results: recall@100 = 0.2533, ndcg@100 = 0.1825, map@100 = 0.0579
"""

'\ntop-5 results: recall@5 = 0.1559, ndcg@5 = 0.1613, map@5 = 0.1076\ntop-10 results: recall@10 = 0.1513, ndcg@10 = 0.1547, map@10 = 0.0851\ntop-20 results: recall@20 = 0.1477, ndcg@20 = 0.1473, map@20 = 0.0669\ntop-50 results: recall@50 = 0.1819, ndcg@50 = 0.1553, map@50 = 0.0562\ntop-100 results: recall@100 = 0.2533, ndcg@100 = 0.1825, map@100 = 0.0579\n'